<a href="https://colab.research.google.com/github/LuigiSigillo/Aviation-accidents-visualizations/blob/main/Data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data preprocessing

before preprocessing

In [ ]:
import sys
import time
import os
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
#!unzip "/content/drive/My Drive/AviationData.csv.zip" -d "/content/drive/My Drive/"

df = pd.read_csv("/content/drive/My Drive/AviationData.csv", encoding='ISO-8859-1')
df

# After preprocessing

In [ ]:
import numpy as np
df = pd.read_csv("/content/drive/My Drive/AviationData.csv", encoding='ISO-8859-1')

#View the dataset
print("1, intero", df.shape, "TOT = ", df.shape[0]*df.shape[1])



df = df.drop(
    columns=["Accident.Number","Airport.Code","Airport.Name","Registration.Number","Engine.Type","FAR.Description","Schedule","Air.Carrier","Report.Status","Publication.Date"]
)
print("2, senza colonne inutili", df.shape, "TOT = ", df.shape[0]*df.shape[1])



print("__________________________________________")
df = df.drop(df[df["Investigation.Type"] == "Incident"].index)
df=df.drop(columns=["Investigation.Type"])

print("3, senza incidents", df.shape, "TOT = ", df.shape[0]*df.shape[1])



print("__________________________________________")
df = df[df["Latitude"].notnull()]
df = df[df["Longitude"].notnull()]

print("4, solo coordinate", df.shape, "TOT = ", df.shape[0]*df.shape[1])


print("__________________________________________")
## Da mettere solo quelli con fatal (#num morti) o non fatal
patternDel = ".*Fatal.*"
filter = df["Injury.Severity"].str.contains(patternDel)
df = df[filter]

print("5, solo severity", df.shape, "TOT = ", df.shape[0]*df.shape[1])


print("__________________________________________")
df = df[df["Aircraft.Damage"].notnull()]
print("6, solo damage", df.shape, "TOT = ", df.shape[0]*df.shape[1])
df=df.drop(columns=["Aircraft.Category"])

'''
print("__________________________________________")
df = df[df["Aircraft.Category"]=="Airplane"]

print("7, solo airplane", df.shape, "TOT = ", df.shape[0]*df.shape[1])

print("__________________________________________")
df = df[df["Amateur.Built"]=="No"]

print("8, solo non amatoriali", df.shape, "TOT = ", df.shape[0]*df.shape[1])
'''
df=df.drop(columns=["Amateur.Built"])
print("__________________________________________")
df = df[df["Purpose.of.Flight"].notnull()]
df = df[df["Purpose.of.Flight"]!="Unknown"]
print("9, solo con purpose", df.shape, "TOT = ", df.shape[0]*df.shape[1])

print("__________________________________________")
df = df.dropna(subset=['Total.Fatal.Injuries','Total.Serious.Injuries'], how='all')
print("10, tutti NAN a casa", df.shape, "TOT = ", df.shape[0]*df.shape[1])


print("__________________________________________")
df = df[df["Weather.Condition"].notnull()]
print("11, solo con wether", df.shape, "TOT = ", df.shape[0]*df.shape[1])

print("__________________________________________")
df = df[df["Broad.Phase.of.Flight"].notnull()]
print("12, solo board phase", df.shape, "TOT = ", df.shape[0]*df.shape[1])


print("__________________________________________")
df = df[df["Number.of.Engines"].notnull()]
df=df.drop(columns=["Number.of.Engines"])

print("13, solo motore", df.shape, "TOT = ", df.shape[0]*df.shape[1])


print("__________________________________________")
df = df[df["Model"].notnull()]
df = df[df["Make"].notnull()]
print("14, not nan make e model", df.shape, "TOT = ", df.shape[0]*(df.shape[1]-3))



print("__________________________________________")
df =  df.drop(df[df["Country"] != "United States"].index)
df=df.drop(columns=["Country"])
print("15, only USA", df.shape, "TOT = ", df.shape[0]*(df.shape[1]-3))



df=df.drop(columns=["Injury.Severity"])
df=df.drop(columns=["Model"])
df["Crash.Location"] = df["Latitude"].astype(str) + "," + df["Longitude"].astype(str) 
df=df.drop(columns=["Latitude"])
df=df.drop(columns=["Longitude"])
df['Total.Fatal.Injuries'] = df['Total.Fatal.Injuries'].fillna(0)
df = df.fillna(0)


"""
print("__________________________________________")
df = df.drop(df[df["Injury.Severity"] == "Non-Fatal"].index)
df = df.drop(df[df["Injury.Severity"] == "Unavailable"].index)
print("16, solo fatali", df.shape, "TOT = ", df.shape[0]*df.shape[1]-3)
print(df.head())
"""
"""
print("__________________________________________")
df = df.drop(df[df["Event.Date"] < "2015-01-01"].index)
df = df.drop(df[df["Injury.Severity"] == "Unavailable"].index)
print("4, dal 2000 in poi", df.shape, "TOT = ", df.shape[0]*(df.shape[1]-3))
print(df.head())
"""


print("TOT = ", df.shape[0]*(df.shape[1]))
#df.to_csv(r'/content/drive/My Drive/Aviation cleaned.csv', index = False)
df.describe()
df

# Coordinates to Country

In [ ]:
import requests
from bs4 import BeautifulSoup
apiurl = "http://api.geonames.org/countrySubdivision?"
#lat=42.561944&lng=-71.77166700000001&username=LENZA
lat = "&lat="
lng = "&lng="
user = "&username=LENZA"




for index, row in df.iterrows():
    latlng = row['Crash.Location']

    print(str(index) + " | " + row['Crash.Location'])
    # index -> indice di riga
    # row['Crash.Location'] -> latitudine,longitudine
    latitude = latlng.split(",")[0]
    longitude = latlng.split(",")[1]
    #print(latitude + " | " + longitude)
    r = requests.get(apiurl + lat + latitude + lng + longitude + user)
    print(r)
    soup = BeautifulSoup(r.content, 'html.parser')
    #print(soup.prettify())
    try:
        print(soup.adminname1.string)
        df.loc[index, 'Crash.Country'] = soup.adminname1.string
    except:
        df.loc[index, 'Crash.Country'] = "Unknown"
    #df[index]['Crash.Country'] = soup.adminname1.string
    

df


In [ ]:
df

In [ ]:
print("TOT = ", df.shape[0]*(df.shape[1]))
df.to_csv(r'/content/drive/My Drive/AviationCrashLocation_new.csv', index = False)
df.describe()
df